In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (8,6)
from sklearn.model_selection import train_test_split
import pylab 
import time
from tqdm import tqdm

In [4]:
df_loc = pd.read_csv("../synthetic_data_simulations/2d_nongaussian_1200_1.csv", sep = ",")

# N = len(df_loc)
# s = np.vstack((df_loc["x"],df_loc["y"])).T


# y = np.array(np.array(df_loc[["var1","var2"]]))



In [5]:
# df_loc = pd.read_csv("../train_data_nongaussian.csv", sep = ",")

N = len(df_loc)
s = np.vstack((df_loc["x"],df_loc["y"])).T


y = np.array(np.array(df_loc[["var1","var2"]]))

In [8]:
num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [9]:
s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi, y, 
                                                                                    test_size=0.1)
print(s_test.shape)
s_train_ensemble, s_train_mse, X_train_ensemble, X_train_mse, y_train_ensemble, y_train_mse= train_test_split(s_train, encoder_train, y_train, test_size=0.1)
print(s_train_ensemble.shape)

(120, 2)
(972, 2)


In [10]:
# model layers 

input_dim = Input(shape = (phi.shape[1], ))
layer1 = Dense(100, kernel_initializer='he_uniform', activation = 'relu')(input_dim)
layer2 = Dense(100, activation = 'relu')(layer1)
layer3 = Dense(100, activation = 'relu')(layer2)
layer4 = Dense(50, activation = 'relu')(layer3)
layer5 = Dense(50, activation = 'relu')(layer4)
final_layer = Dense(2, activation = 'linear')(layer5)

In [11]:
s_train_ensemble1, s_train_ensemble2, X_train_ensemble1, X_train_ensemble2, y_train_ensemble1, y_train_ensemble2= train_test_split(s_train_ensemble, X_train_ensemble, y_train_ensemble, test_size=0.2)
print(s_train_ensemble1.shape)
base_model = Model(inputs = input_dim, outputs = final_layer)
optimizer = keras.optimizers.Adam(lr=0.01)
# Compile the Model
base_model.compile(optimizer = optimizer, loss = 'mae')
base_model.summary()
base_model.fit(X_train_ensemble1, y_train_ensemble1,validation_split = 0.1, epochs = 400,
                batch_size = 64,verbose = 2)

(777, 2)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 110)]             0         
_________________________________________________________________
dense (Dense)                (None, 100)               11100     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 2)              

Epoch 120/400
11/11 - 0s - loss: 0.2939 - val_loss: 0.7083
Epoch 121/400
11/11 - 0s - loss: 0.3414 - val_loss: 0.6975
Epoch 122/400
11/11 - 0s - loss: 0.3616 - val_loss: 0.8074
Epoch 123/400
11/11 - 0s - loss: 0.3380 - val_loss: 0.6635
Epoch 124/400
11/11 - 0s - loss: 0.3589 - val_loss: 0.6604
Epoch 125/400
11/11 - 0s - loss: 0.3317 - val_loss: 0.6656
Epoch 126/400
11/11 - 0s - loss: 0.3212 - val_loss: 0.6682
Epoch 127/400
11/11 - 0s - loss: 0.3619 - val_loss: 0.6904
Epoch 128/400
11/11 - 0s - loss: 0.3534 - val_loss: 0.6582
Epoch 129/400
11/11 - 0s - loss: 0.3343 - val_loss: 0.6462
Epoch 130/400
11/11 - 0s - loss: 0.3722 - val_loss: 0.7740
Epoch 131/400
11/11 - 0s - loss: 0.3606 - val_loss: 0.6294
Epoch 132/400
11/11 - 0s - loss: 0.3615 - val_loss: 0.5999
Epoch 133/400
11/11 - 0s - loss: 0.3618 - val_loss: 0.8263
Epoch 134/400
11/11 - 0s - loss: 0.3165 - val_loss: 0.6824
Epoch 135/400
11/11 - 0s - loss: 0.2999 - val_loss: 0.6940
Epoch 136/400
11/11 - 0s - loss: 0.2980 - val_loss: 0.67

Epoch 259/400
11/11 - 0s - loss: 0.3123 - val_loss: 0.7108
Epoch 260/400
11/11 - 0s - loss: 0.3045 - val_loss: 0.6525
Epoch 261/400
11/11 - 0s - loss: 0.2719 - val_loss: 0.7904
Epoch 262/400
11/11 - 0s - loss: 0.2590 - val_loss: 0.7121
Epoch 263/400
11/11 - 0s - loss: 0.2542 - val_loss: 0.7227
Epoch 264/400
11/11 - 0s - loss: 0.2452 - val_loss: 0.7055
Epoch 265/400
11/11 - 0s - loss: 0.2197 - val_loss: 0.7441
Epoch 266/400
11/11 - 0s - loss: 0.2849 - val_loss: 0.7701
Epoch 267/400
11/11 - 0s - loss: 0.2887 - val_loss: 0.7228
Epoch 268/400
11/11 - 0s - loss: 0.2742 - val_loss: 0.7427
Epoch 269/400
11/11 - 0s - loss: 0.2394 - val_loss: 0.7347
Epoch 270/400
11/11 - 0s - loss: 0.2493 - val_loss: 0.7544
Epoch 271/400
11/11 - 0s - loss: 0.2011 - val_loss: 0.7328
Epoch 272/400
11/11 - 0s - loss: 0.1939 - val_loss: 0.7214
Epoch 273/400
11/11 - 0s - loss: 0.2046 - val_loss: 0.6992
Epoch 274/400
11/11 - 0s - loss: 0.1979 - val_loss: 0.6908
Epoch 275/400
11/11 - 0s - loss: 0.2026 - val_loss: 0.67

Epoch 398/400
11/11 - 0s - loss: 0.1610 - val_loss: 0.7088
Epoch 399/400
11/11 - 0s - loss: 0.1604 - val_loss: 0.7151
Epoch 400/400
11/11 - 0s - loss: 0.1705 - val_loss: 0.7775


In [12]:
# defining the base model
base_model1 = Model(inputs = input_dim, outputs = layer3)

def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae
# define and fit the model
def fit_model(X_train, y_train,X_test,y_test):
    # DeepKriging model for continuous data
    ensemble_model = Sequential()

    ensemble_model.add(base_model1)
#     ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(2, activation='linear'))
    ensemble_model.layers[-3].trainable = False
    optimizer = keras.optimizers.Adam(lr=0.01)
    ensemble_model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])
    ensemble_model.fit(X_train, y_train,
                       validation_data=(X_test,y_test), epochs = 200, batch_size = 64, verbose = 0)
    return ensemble_model

# fit an ensemble of models
def fit_ensemble(n_members, X_train, Y_train):
    ensemble = list()
    for i in range(n_members):
        x_train, x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size = 0.2)
        # define and fit the model on the training set
        model = fit_model(x_train, y_train,x_test,y_test)
        # evaluate model on the test set
        yhat = model.predict(x_test, verbose=0)
        mae1 = mae(yhat, y_test)
        print('>%d, MAE: %.3f' % (i+1, mae1))
        # store the model
        ensemble.append(model)
    return ensemble
def y_list_uni(yhat,i):
    the_list = list()
    for j in range(len(yhat)):
        the_list.append(yhat[j][0][i])
    return the_list
# make predictions with the ensemble and calculate a prediction interval
def variance(data):
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
    # Variance
    variance = sum(deviations) / (n-1)
    return variance
def predict_with_pi(ensemble, X):
    mean_vec1 = list()
    mean_vec2 = list()
    var_vec1 = list()
    var_vec2 = list()
    # make predictions
    yhat = [model.predict(X, verbose=0) for model in ensemble]
    for data in tqdm(range(X.shape[0])):
        yhat1 = np.asarray(np.asarray(yhat)[:,data,:])
        var1_pred = yhat1[:,0]
        var2_pred = yhat1[:,1]
        mean1 = var1_pred.mean()
        var1 = variance(var1_pred)
        mean2 = var2_pred.mean()
        var2 = variance(var2_pred)
        mean_vec1.append(mean1)
        mean_vec2.append(mean2)
        var_vec1.append(var1)
        var_vec2.append(var2)
    return mean_vec1, var_vec1, mean_vec2, var_vec2

In [14]:
# fit ensemble
# start_time = time.time()
n_members = 20
ensemble = fit_ensemble(n_members, X_train_ensemble, y_train_ensemble)

# train data mean and variance vectors
mean_vec1, var_vec1, mean_vec2, var_vec2 = predict_with_pi(ensemble, X_train_mse)

>1, MAE: 0.339
>2, MAE: 0.538
>3, MAE: 0.733
>4, MAE: 0.619
>5, MAE: 0.689
>6, MAE: 0.552
>7, MAE: 0.341
>8, MAE: 0.499
>9, MAE: 0.380
>10, MAE: 0.519
>11, MAE: 0.773
>12, MAE: 0.541
>13, MAE: 0.805
>14, MAE: 0.714
>15, MAE: 0.703
>16, MAE: 0.677
>17, MAE: 0.481
>18, MAE: 0.447
>19, MAE: 0.507
>20, MAE: 0.570


100%|██████████| 108/108 [00:00<00:00, 1390.71it/s]


In [15]:
# random error calculation on training data
r1 = list()
r2 = list()
r1 = (y_train_mse[:,0] - mean_vec1)**2 - var_vec1
for i in range(len(r1)):
    if r1[i] < 0: r1[i] = 0.0
r2 = (y_train_mse[:,1] - mean_vec2)**2 - var_vec2
for i in range(len(r2)):
    if r2[i] < 0: r2[i] = 0.0
        
def calc_distance(s1,s2):
    return(np.sqrt((s1[0] - s2[0])**2 + (s1[1] - s2[1])**2))

def get_nearest_data(s_train,s_test,r,k):
    dist_mat = np.zeros((len(s_test),len(s_train)))
    nearest_var = np.zeros((len(s_test)))
    for i in range(len(s_train)):
        for j in range(len(s_test)):
            dist_mat[j][i] = calc_distance(s_train[i],s_test[j])
    for i in range(len(s_test)):
        a = dist_mat[i]
        b = np.argpartition(a,k)[:k]
        y_val = []
        for index in b:
            y_val.append(r[index])
        nearest_var[i] = (np.mean(y_val))
    return nearest_var

In [16]:
### Test data 

# df_test = pd.read_csv("../synthetic_datasets/test_data_nongaussian.csv", sep = ",")
# s_test = np.vstack((df_test["x"],df_test["y"])).T
# y_test = np.array(df_test[["var1","var2"]])

# # mean and variance vector for prediction data
# N = len(df_test)
# s = s_test
# num_basis = [2**2,5**2,9**2]
# knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
# #knots_1d = [np.linspace(0,1,i) for i in num_basis]
# ##Wendland kernel
# K = 0
# phi = np.zeros((N, sum(num_basis)))
# for res in range(len(num_basis)):
#     theta = 1/np.sqrt(num_basis[res])*2.5
#     knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
#     knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
#     for i in range(num_basis[res]):
#         d = np.linalg.norm(s-knots[i,:],axis=1)/theta
#         for j in range(len(d)):
#             if d[j] >= 0 and d[j] <= 1:
#                 phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
#             else:
#                 phi[j,i + K] = 0
#     K = K + num_basis[res]
# encoder_test = phi

In [17]:
N

1200

In [18]:
# random error calculation on test data with neighbourhood approach
r1_pred = get_nearest_data(s_train_mse,s_test,r1,20)
r2_pred = get_nearest_data(s_train_mse,s_test,r2,20)

In [19]:
# mean and variance vector for prediction data
mean_vec1, var_vec1, mean_vec2, var_vec2 = predict_with_pi(ensemble, encoder_test)

100%|██████████| 120/120 [00:00<00:00, 1375.46it/s]


In [20]:
# e = list()
# for i in range(len(r1_pred)):
#     e.append(r1_pred[i][0])
# print(len(var_vec1))
# error_vec2 = model_mse2.predict(encoder_test)
var_vec1 = np.asarray(var_vec1)
var_vec2 = np.asarray(var_vec2)
# e = np.asarray(e)
lower_bound1 = mean_vec1 - 1.96*np.sqrt(var_vec1 + r1_pred)
upper_bound1 = mean_vec1 + 1.96*np.sqrt(var_vec1 + r1_pred)
lower_bound2 = mean_vec2 - 1.96*np.sqrt(var_vec2 + r2_pred)
upper_bound2 = mean_vec2 + 1.96*np.sqrt(var_vec2 + r2_pred)

In [21]:
count_var0 = 0
count_var1 = 0
for i in range(len(y_test)):
    if ((y_test[i,0] > lower_bound1[i]) and (y_test[i,0] < upper_bound1[i])): count_var0 +=1
    if y_test[i,1] > lower_bound2[i] and y_test[i,1] < upper_bound2[i]: count_var1 +=1

In [22]:
picp1 = count_var0/len(y_test)
picp2 = count_var1/len(y_test)
print(str(picp1)+","+str(picp2))

1.0,0.9416666666666667


In [23]:
width1 = np.mean(upper_bound1 - lower_bound1)
width2 = np.mean(upper_bound2 - lower_bound2)
print(str(width1)+","+str(width2))

5.927518840743226,1.8381904810108123


In [24]:
df_bounds = pd.DataFrame(np.vstack((lower_bound1,upper_bound1,lower_bound2,upper_bound2)).T,
                         columns = ["l1","u1","l2","u2"])
df_bounds.to_csv("../plot_results/bounds_nongaussian.csv")